# AI Shorts Generator — Google Colab + Gradio

Beginner-friendly UI to: upload a long video (or paste YouTube URL), optionally add an SRT, choose aspect ratio and clip options, burn subtitles, add watermark/logo, auto-generate titles, and download a ZIP with everything. No .env editing — enter API keys directly in the UI.

Usage:
1. Run the Setup cell.
2. Run the UI cell and click the public Gradio link.
3. Fill options and press ‘Generate Shorts’.
4. When done, download the Results ZIP.


In [ ]:
# Setup: system packages and Python dependencies
!apt-get update -qq
!apt-get install -y -qq ffmpeg imagemagick fonts-freefont-ttf > /dev/null
!sed -i 's/<policy domain="path" rights="none" pattern="@\*" />/<!-- <policy domain="path" rights="none" pattern="@\*" /> -->/g' /etc/ImageMagick-6/policy.xml || true
%pip -q install --upgrade pip wheel setuptools
%pip -q install gradio==4.* moviepy==1.0.3 imageio-ffmpeg
%pip -q install numpy<2.0 opencv-python-headless pytubefix pydub pysrt faster-whisper google-generativeai
%pip -q install 'openai>=1.35.0'

import torch, os
if torch.cuda.is_available():
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    %pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else:
    %pip -q install torch torchvision torchaudio

!mkdir -p models
!wget -q -O models/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
print('Setup complete. CUDA:', torch.cuda.is_available())


In [ ]:
# Gradio UI
import gradio as gr
from pipeline_advanced import generate_pipeline

def run_ui():
    with gr.Blocks() as demo:
        gr.Markdown('# AI Shorts Generator')
        with gr.Row():
            with gr.Column(scale=1):
                youtube_url = gr.Textbox(label='YouTube URL (optional)')
                video_file = gr.File(label='Or upload a video')
                srt_file = gr.File(label='Upload SRT (optional)')
                provider = gr.Dropdown(['OpenAI','Gemini'], label='AI Provider', value='OpenAI')
                openai_key = gr.Textbox(label='OpenAI API Key', type='password', visible=True)
                gemini_key = gr.Textbox(label='Gemini API Key', type='password', visible=False)
                target_len = gr.Slider(10, 120, value=60, step=1, label='Target short length (s)')
                tol = gr.Slider(2, 30, value=10, step=1, label='Length tolerance ± (s)')
                max_clips = gr.Slider(1, 10, value=3, step=1, label='Maximum number of clips')
                aspect = gr.Dropdown(['9:16','16:9','1:1'], value='9:16', label='Aspect ratio')
                crop_mode = gr.Dropdown(['Center','Face-track'], value='Face-track', label='Crop mode')
                karaoke = gr.Checkbox(label='Burn subtitles (karaoke) into output videos', value=True)
                export_srt = gr.Checkbox(label='Export SRTs + full transcription in ZIP', value=True)
                title_mode = gr.Dropdown(['Auto','Custom','None'], value='Auto', label='Title overlay mode')
                custom_title = gr.Textbox(label='Custom title (if mode = Custom)')
                platform = gr.Dropdown(['TikTok','YouTube','Generic'], value='TikTok', label='Platform (safe margins)')
                out_prefix = gr.Textbox(label='Output name prefix', value='short')
                watermark_file = gr.File(label='Watermark/logo image (optional)')
                seo_text = gr.Textbox(label='Custom description / SEO tags (optional)', lines=3)
                go = gr.Button('Generate Shorts', variant='primary')
            with gr.Column(scale=1):
                logs = gr.Textbox(label='Process logs', lines=20)
                out_zip = gr.File(label='Results ZIP')
        def _toggle(p):
            return (gr.update(visible=p=='OpenAI'), gr.update(visible=p=='Gemini'))
        provider.change(_toggle, inputs=provider, outputs=[openai_key, gemini_key])
        
        def _run(youtube_url, video_file, srt_file, provider, openai_key, gemini_key, target_len, tol, max_clips, aspect, crop_mode, karaoke, export_srt, title_mode, custom_title, platform, out_prefix, watermark_file, seo_text):
            logs_buf = []
            def log(m):
                logs_buf.append(str(m))
            min_len = max(5, int(target_len) - int(tol))
            max_len = int(target_len) + int(tol)
            try:
                zip_path = generate_pipeline(youtube_url, video_file, srt_file, provider, openai_key, gemini_key, min_len, max_len, int(max_clips), aspect, crop_mode, bool(karaoke), bool(export_srt), title_mode, custom_title, platform, out_prefix, watermark_file, seo_text, logger=log)
                log('Done.' if zip_path else 'Failed to generate.')
                return zip_path, '\n'.join(logs_buf)
            except Exception as ex:
                log(f'Error: {ex}')
                return None, '\n'.join(logs_buf)
        
        go.click(_run, [youtube_url, video_file, srt_file, provider, openai_key, gemini_key, target_len, tol, max_clips, aspect, crop_mode, karaoke, export_srt, title_mode, custom_title, platform, out_prefix, watermark_file, seo_text], [out_zip, logs])
    return demo

demo = run_ui()
demo.launch(debug=True, share=True)
